## MYSQL에 연결해서 아티스트 및 노래 제목 json 만들기

In [1]:
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 13.1 MB/s eta 0:00:000:00:01m eta 0:00:01


In [3]:
import mysql.connector
import json

def fetch_unique_song_and_artist_data():
    try:
        # MySQL 연결 설정
        connection = mysql.connector.connect(
            host='www.dungdungcloud.shop',         # MySQL 호스트 이름 또는 IP 주소
            user='root',     # MySQL 사용자 이름
            password='1234', # MySQL 비밀번호
            database='singsong_db'    # 사용할 데이터베이스 이름
        )

        if connection.is_connected():
            cursor = connection.cursor()

            # SQL 쿼리 작성: song_info 테이블에서 song_name, artist_name의 유니크한 값 가져오기
            query = "SELECT DISTINCT song_name, artist_name FROM song_info"
            cursor.execute(query)

            # 결과 가져오기
            results = cursor.fetchall()

            # 중복을 피하기 위해 세트 사용
            unique_songs = set()
            unique_artists = set()

            # 결과를 반복하며 리스트에 추가
            for row in results:
                song_name = row[0]
                artist_name = row[1]
                if song_name:
                    unique_songs.add(song_name)
                if artist_name:
                    unique_artists.add(artist_name)

            # 리스트로 변환
            known_songs_list = list(unique_songs)
            known_artists_list = list(unique_artists)

            # JSON 파일로 저장
            data = {
                "songs": known_songs_list,
                "artists": known_artists_list
            }

            with open('known_songs_and_artists.json', 'w', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent=4)

            print("JSON 파일 저장 완료: known_songs_and_artists.json")
            return known_songs_list, known_artists_list

    except mysql.connector.Error as e:
        print(f"MySQL 오류: {e}")
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL 연결이 종료되었습니다.")

# 함수 호출
known_songs_list, known_artists_list = fetch_unique_song_and_artist_data()

# 결과 출력
print("Known Artists List: ", known_artists_list)
print("Known Songs List: ", known_songs_list)

JSON 파일 저장 완료: known_songs_and_artists.json
MySQL 연결이 종료되었습니다.
Known Artists List:  ['백지영,이홍기', '윤희상', '효린(Feat.크루셜스타)', '비와이(Feat.크러쉬)', '더 라임', '이센스(Feat.Uneducated Kid)', 'ตอง ภัครมัย โปตระนันท์', 'CESAR COSTA', '五木ひろし & マリエ', 'STEVE CONTE', '홍서범', 'Rie fu', '빌리 어코스티', '倖田来未', 'T.T.MA', '川本眞琴', 'Meat Loaf', '이문세(Feat.박인수)', '명희', '姜志', '김재하', 'Jewelry', 'Busted', '정화', 'ЛОБОДА СВЕТЛАНА', '엘(인피니트),박재정', 'Lemonade', 'Văn Bình-Hồng Hạnh', 'HAWAIIAN6', '린(With 봉구)', 'JM', 'Jet', '윤민수(With 박정은,4MEN)', 'Masta WU(Feat.Dok2,바비)', '설아(우주소녀)', '아자', '황진희', 'Tommy Bolin', '기리보이(Feat.최정윤)', '키(KEY)(Feat.소유)', 'SHU-I', '텐(TEN)', '빅뱅(Feat.노브레인)', 'FLOW×GRANRODEO', '티키틱(Feat.김세정)', '이승환,김예림(투개월)', 'Sik-K,릴보이,지구인(Feat.로꼬,그레이,DJ Pumkin)', '7!!', '예솔이,이규대', 'Trương Quốc Khánh', '페퍼톤스(With 옥상달빛)', '커피소년(Feat.박현종)', 'Usher&Monica', '원티드', '황소윤,수민(SUMIN)', '김태훈', '김준(Feat.김조한)', '박윤창', '愛内里菜', '김소희', 'Irvan Mansyur S', 'N9', '크루셜스타(Feat.Bob James,메이슨더소울)', '岡 晴夫', '다이나믹듀오(Feat.팔로알토,나플라,첸슬러)', '范玮琪,张韶涵',

## 초기 

In [20]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai.chat_models import ChatOpenAI
from langchain.chains import SequentialChain, LLMChain
from langchain.chains.transform import TransformChain
from langchain_core.output_parsers import StrOutputParser
from langchain_milvus import Milvus
import logging
import os
from langchain_core.callbacks import StreamingStdOutCallbackHandler
from langchain_openai import OpenAIEmbeddings


OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

embedding_model = OpenAIEmbeddings(
    model="text-embedding-3-large",
)

llm = ChatOpenAI(
    temperature=0.5,
    max_tokens=4096,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    model_name='gpt-4o-mini',
    api_key=OPENAI_API_KEY
)

milvus = Milvus(
    embedding_function=embedding_model,
    collection_name="singsongsangsong_22286",
    connection_args={"host": "localhost", "port": "19530"},
    text_field="song_name"
)

# Prompt 1: 아티스트 이름과 노래 제목 추출
prompt1 = ChatPromptTemplate.from_template(
    """
    Extract the artist name and song name from the following query if exists: {query}.
    Return the result in the following format:
    Artist: <artist_name>, Song: <song_name>

    if the artist name and song name are not found
    the result in the following format:
    Artist: Unknown, Song: Unknown
    """
)

# Prompt 2: 검색된 메타데이터를 바탕으로 구체적인 쿼리 생성
prompt2 = ChatPromptTemplate.from_template(
    """
    You are an expert music recommendation assistant. A user is looking for songs based on the following query: 
    \"{query}\". 

    The vector collection contains songs with metadata like genre, era, mood, singer type, country, and special contexts such as 'Between Exciting and Calm mood', 'Break up with someone who I care a lot', 'reminiscence of the past', 'Bass songs that are easy', 'I want to Rest', 'Dance Actively', 'celebrate the wedding', and others.

    The user mentioned specific songs or artists: 
    {song_metadata_str}

    Please rewrite the query to make it more specific, including details about the song’s era, mood, and potential use case, so it can match the songs from the collection more accurately.

    Respond in the following format: 
    FIXED SENTENCE: <Rewrite the query with more specific details here>
"""
)

# Step 1: Milvus DB에서 song_name과 artist_name을 LIKE 문으로 검색
def search_milvus_with_filter(artist_name, song_title):
    try:
        # SQL-like 검색 표현식 구성
        query_expr = f'artist_name LIKE "%{artist_name}%" AND song_name LIKE "%{song_title}%"'

        # SQL-like 검색을 통해 필터링된 데이터를 검색
        results = milvus.similarity_search(
            query=f"{artist_name} released the song '{song_title}'",
            k=5,
            search_kwargs={"expr": query_expr}  # SQL-like 표현식 적용
        )
        
        if results:
            return "\n".join([f"Song: {res.page_content}, Artist: {res.metadata['artist_name']}, Genre: {res.metadata.get('genre', 'Unknown')}, Singer Type: {res.metadata.get('singer_type', 'Unknown')}" for res in results])
        else:
            return "No matching songs found in the database."

    except Exception as e:
        logging.error(f"Error searching Milvus: {e}")
        return "Error retrieving data from Milvus."

# Step 2: 검색된 메타데이터로 프롬프트에 활용할 쿼리 생성
def construct_metadata_string(output):
    # output에서 'Artist: <artist_name>, Song: <song_name>' 형식으로 추출
    artist_name, song_title = "", ""
    if 'Artist:' in output and 'Song:' in output:
        parts = output.split(',')
        artist_name = parts[0].split('Artist:')[-1].strip()
        song_title = parts[1].split('Song:')[-1].strip()
    
    # Milvus에서 아티스트 이름과 노래 제목으로 검색
    metadata_string = search_milvus_with_filter(artist_name, song_title)
    print("\n=== Song Metadata ===")
    print(metadata_string)
    print("\n")
    
    # Milvus 검색 결과 반환
    return {"song_metadata_str": metadata_string}  # dict 형식으로 반환

# LLMChain으로 감싸기
llm_chain1 = LLMChain(prompt=prompt1, llm=llm, output_key="artist_and_song")
llm_chain2 = LLMChain(prompt=prompt2, llm=llm, output_key="fixed_sentence")

# TransformChain: Step 1과 Step 2를 연결
transform_chain = TransformChain(
    input_variables=["artist_and_song"],  # llm_chain1의 결과에서 'artist_and_song'을 사용
    output_variables=["song_metadata_str"],
    transform=construct_metadata_string  # 수정된 함수
)

# SequentialChain: prompt1 -> transform_chain -> prompt2로 진행
sequential_chain = SequentialChain(
    chains=[
        llm_chain1,  # 첫 번째 LLM 실행
        transform_chain,  # Milvus에서 메타데이터 가져오기
        llm_chain2  # 두 번째 LLM 실행
    ],
    input_variables=["query"],
    output_variables=["fixed_sentence"]
)

# 사용자 입력 처리
user_query = "버즈의 가시와 비슷한 노래를 추천해줘"

# SequentialChain을 실행하여 전체 프로세스를 완료
response = sequential_chain.invoke({"query": user_query})

# 결과 출력
# song_metadata_str 및 두 번째 프롬프트 출력 확인
song_metadata_str = response["song_metadata_str"] if "song_metadata_str" in response else "No metadata found"
fixed_sentence = response["fixed_sentence"]

Artist: 버즈, Song: 가시
=== Song Metadata ===
Song: The Art Of Letting Go, Artist: Mikaila, Genre: , Singer Type: 
Song: Let Your Hair Down, Artist: Magic!, Genre: 포크, Singer Type: 그룹
Song: right here, Artist: keshi, Genre: POP, Singer Type: 남성, 솔로
Song: beside you, Artist: keshi, Genre: POP, Singer Type: 남성, 솔로
Song: Let Somebody Go, Artist: Coldplay,Selena Gomez, Genre: POP, Singer Type: 


FIXED SENTENCE: "버즈의 가시와 비슷한 노래를 추천해줘. 감정적으로 깊고 서정적인 발라드 장르의 곡으로, 이별의 아픔을 느끼며 차분한 분위기를 원해. 2000년대 초반의 곡이면 좋겠어."

In [29]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai.chat_models import ChatOpenAI
from langchain.chains import SequentialChain, LLMChain
from langchain.chains.transform import TransformChain
from langchain_core.output_parsers import StrOutputParser
from langchain_milvus import Milvus
import logging
import os
from langchain_core.callbacks import StreamingStdOutCallbackHandler
from langchain_openai import OpenAIEmbeddings

# Milvus 설정
milvus = Milvus(
    embedding_function=embedding_model,
    collection_name="singsongsangsong_22286",
    connection_args={"host": "localhost", "port": "19530"},
    text_field="song_name"
)

# Milvus retriever 설정
retriever = milvus.as_retriever()

# LLM 설정
llm = ChatOpenAI(
    temperature=0.5,
    model_name='gpt-4o-mini',
    api_key=OPENAI_API_KEY
)

# 1. 사용자 입력에서 가수 이름과 노래 제목 추출하는 프롬프트
prompt_extract = ChatPromptTemplate.from_template(
    """
    Extract the artist name and song title from the following query: {query}.
    If the artist name and song title are found, return the result in the following format:
    Artist: <artist_name>, Song: <song_title>.
    """
)

# 2. Milvus DB에서 추출된 정보를 검색하는 함수 (retriever 사용, 여러 가수와 노래 제목 처리)
def search_milvus_with_retriever(output):
    extracted_info = output["extracted_info"]

    # 여러 가수와 노래 제목이 포함된 경우를 처리 (리스트로 분리)
    artist_song_pairs = extracted_info.split(", ")  # 각 가수와 노래 제목을 분리
    search_results = []

    for pair in artist_song_pairs:
        # 각각의 pair는 "Artist: <artist_name>, Song: <song_title>" 형식이므로 이를 분리
        artist_name = pair.split("Artist:")[-1].split(",")[0].strip()
        song_title = pair.split("Song:")[-1].strip()

        # 각 가수와 노래 제목에 대해 Milvus DB에서 검색 수행
        search_query = f"{artist_name} released the song '{song_title}'"
        results = retriever.get_relevant_documents(search_query)

        # 검색 결과를 리스트에 추가
        search_results.append(results)

    # 여러 검색 결과를 하나로 합치기
    return {"retrieved_data": search_results}

# 3. 검색된 정보를 바탕으로 LLM이 구체적인 답변 생성
prompt_refine = ChatPromptTemplate.from_template(
    """
    Based on the query "{query}" and the following song metadata retrieved from the database:
    {retrieved_data}

    Please provide a detailed music recommendation including songs not only from the same artist, but also from other artists with similar styles, eras, and genres.
    """
)

# 4. LLMChain을 통해 사용자 입력에서 가수 이름과 노래 제목을 추출
llm_chain1 = LLMChain(llm=llm, prompt=prompt_extract, output_key="extracted_info")

# 5. TransformChain을 통해 Milvus에서 검색 수행
transform_chain = TransformChain(
    input_variables=["extracted_info"],
    output_variables=["retrieved_data"],
    transform=search_milvus_with_retriever
)

# 6. 검색된 데이터를 바탕으로 LLM이 추천 답변 생성
llm_chain2 = LLMChain(llm=llm, prompt=prompt_refine, output_key="final_response")

# 7. SequentialChain으로 전체 작업을 순차적으로 수행
sequential_chain = SequentialChain(
    chains=[llm_chain1, transform_chain, llm_chain2],
    input_variables=["query"],
    output_variables=["final_response"]
)

# 사용자 입력 처리
user_query = "버즈의 가시, 박효신의 동경 이라는 노래와 비슷한 노래를 추천해줘"

# SequentialChain 실행
response = sequential_chain.invoke({"query": user_query})

# 결과 출력
print(response["final_response"])

Based on your request for a song that fits a "완전 신나는 느낌의 마지막곡에 어울리는 노래" (a song that matches a completely exciting feeling for the last track), here are some recommendations from various artists across similar styles, eras, and genres:

### Recommendations:

1. **비 (Rain) - "나"**
   - **Year:** 2017
   - **Genre:** R&B/Soul
   - **Description:** This song features a moderate tempo with high energy and dynamic sound textures, making it a great choice for a lively atmosphere. The prominent percussive elements and bright timbre add to its excitement.
   - **Listen Here:** [비 - 나](https://singsong-audio-bucket.s3.amazonaws.com/62953나.mp3)
   - ![비 - 나 Album Cover](https://cdnimg.melon.co.kr/cm/album/images/100/61/235/10061235_500.jpg?16c9e8bcbccccfee629dc7cb7721a394/melon/resize/282/quality/80/optimize)

2. **Anne-Marie & Niall Horan - "Our Song"**
   - **Year:** 2021
   - **Genre:** Pop
   - **Description:** This duet has a bright and complex timbre with a catchy melody that creates an up

## 최종 LLM Recommend

In [40]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai.chat_models import ChatOpenAI
from langchain_milvus import Milvus
from langchain.chains import SequentialChain, LLMChain
from langchain.chains.transform import TransformChain

# Milvus 설정
milvus = Milvus(
    embedding_function=embedding_model,
    collection_name="singsongsangsong_22286",
    connection_args={"host": "localhost", "port": "19530"},
    text_field="song_name"
)

retriever = milvus.as_retriever()

# LLM 설정
llm = ChatOpenAI(
    temperature=0.5,
    model_name='gpt-4o-mini',
    api_key=OPENAI_API_KEY
)

# 1. 노래 제목과 가수 이름 기반 검색 (유형 1)
def search_by_song_and_artist(output):
    extracted_info = output["extracted_info"]

    # 여러 가수와 노래 제목이 포함된 경우를 처리 (리스트로 분리)
    artist_song_pairs = extracted_info.split(", ")  # 각 가수와 노래 제목을 분리
    search_results = []

    for pair in artist_song_pairs:
        # 각각의 pair는 "Artist: <artist_name>, Song: <song_title>" 형식이므로 이를 분리
        artist_name = pair.split("Artist:")[-1].split(",")[0].strip()
        song_title = pair.split("Song:")[-1].strip()

        # 각 가수와 노래 제목에 대해 Milvus DB에서 검색 수행
        search_query = f"{artist_name} released the song '{song_title}'"
        results = retriever.get_relevant_documents(search_query)

        # 검색 결과를 리스트에 추가
        search_results.append(results)

    # 여러 검색 결과를 하나로 합치기
    return {"retrieved_data": search_results}


# 2. 기분이나 분위기 기반 검색 (유형 2)
def search_by_mood_or_theme(user_query):
    prompt = f"""
    The user input is: "{user_query}". 
    Extract the mood or theme from the input and create a query to search for songs with similar emotional qualities, such as mood, energy level, and use case.

    Your output should follow this format:
    - Refined Query: <refined_query>
    """
    
    refined_query = llm(prompt)
    results = retriever.get_relevant_documents(refined_query.content)
    return {"retrieved_data": results}

# 3. 특정 연도나 특성 기반 검색 (유형 3)
def search_by_specific_feature(user_query):
    prompt = f"""
    The user is looking for songs with the following features: "{user_query}". 
    Create a detailed query that includes era, singer type, and genre to retrieve relevant songs from the database.

    Your output should follow this format:
    - Refined Query: <refined_query>
    """
    
    refined_query = llm(prompt)
    results = retriever.get_relevant_documents(refined_query.content)
    return {"retrieved_data": results}

# LLM을 이용해 입력 타입을 판단하는 함수
def determine_input_type(user_query):
    prompt = f"""
    Classify the user's input into one of the following types:
    1. song_and_artist: If the user is asking for songs similar to specific songs or artists.
    2. mood_or_theme: If the user is describing a mood, theme, or situation to find songs that match those qualities.
    3. specific_feature: If the user is looking for songs based on specific features such as year, genre, or singer type.

    Input: "{user_query}"

    Your output should be in the following format:
    Type: <song_and_artist/mood_or_theme/specific_feature>
    """
    
    result = llm(prompt)
    
    print("\n=== LLM Response ===")
    print(result.content)
    # LLM 응답에서 'Type: ' 부분 제거하고 유형만 추출
    input_type = result.content.lower().replace("type:", "").strip()
    
    return input_type

# 4. LLMChain을 통해 검색 및 추천 수행
prompt_extract = ChatPromptTemplate.from_template(
    """
    Extract the artist name and song title from the following query: {query}.
    If no song or artist is found, try to extract the mood, theme, or other musical characteristics from the query.

    Your output should follow this format:
    - Artist: <artist_name>
    - Song: <song_title> OR Mood/Theme: <mood_or_theme>
    """
)

# 5. 검색 결과 바탕으로 추천 생성
prompt_refine = ChatPromptTemplate.from_template(
    """
    Based on the query "{query}" and the following song metadata retrieved from the database:
    {retrieved_data}

    Please provide exactly 5 song recommendations by their song info IDs, formatted as:

    - Recommendation: [<song_info_id1>, <song_info_id2>, <song_info_id3>, <song_info_id4>, <song_info_id5>]

    Only provide the song info IDs without any additional descriptions or explanations.
    """
)

# LLMChains 설정
llm_chain1 = LLMChain(llm=llm, prompt=prompt_extract, output_key="extracted_info")
llm_chain2 = LLMChain(llm=llm, prompt=prompt_refine, output_key="final_response")

# 추천 목록을 정확히 5곡으로 맞추는 함수
def ensure_five_recommendations(recommendation_list):
    # 추천 곡이 5곡보다 적을 경우, 반복해서 추가하여 5곡을 맞춤
    while len(recommendation_list) < 5:
        recommendation_list.append(recommendation_list[-1])  # 마지막 곡을 반복 추가
    # 추천 곡이 5곡보다 많을 경우, 5곡만 선택
    return recommendation_list[:5]

# 노래 제목과 가수 이름 기반 검색 (유형 1)
def search_by_song_and_artist(output):
    extracted_info = output['extracted_info']
    artist_song_pairs = extracted_info.split(", ")
    search_results = []

    for pair in artist_song_pairs:
        artist_name = pair.split("Artist:")[-1].split(",")[0].strip()
        song_title = pair.split("Song:")[-1].strip()
        search_query = f"{artist_name} released the song '{song_title}'"
        results = retriever.get_relevant_documents(search_query)

        # 검색된 song_info_id를 저장
        search_results += [doc.metadata['song_info_id'] for doc in results]

    search_results = ensure_five_recommendations(search_results)  # 5곡으로 맞추기
    return {"retrieved_data": search_results}

# 기분이나 분위기 기반 검색 (유형 2)
def search_by_mood_or_theme(user_query):
    prompt = f"""
    The user input is: "{user_query}". 
    Extract the mood or theme from the input and create a query to search for songs with similar emotional qualities, such as mood, energy level, and use case.
    """
    refined_query = llm(prompt)
    results = retriever.get_relevant_documents(refined_query.content)
    
    search_results = [doc.metadata['song_info_id'] for doc in results]
    search_results = ensure_five_recommendations(search_results)  # 5곡으로 맞추기
    return {"retrieved_data": search_results}

# 특정 연도나 특성 기반 검색 (유형 3)
def search_by_specific_feature(user_query):
    prompt = f"""
    The user is looking for songs with the following features: "{user_query}". 
    Create a detailed query that includes era, singer type, and genre to retrieve relevant songs from the database.
    """
    refined_query = llm(prompt)
    results = retriever.get_relevant_documents(refined_query.content)
    
    search_results = [doc.metadata['song_info_id'] for doc in results]
    search_results = ensure_five_recommendations(search_results)  # 5곡으로 맞추기
    return {"retrieved_data": search_results}

# 실행 함수
def run_chain(user_query):
    input_type = determine_input_type(user_query)

    if input_type == "song_and_artist":
        transform_chain = TransformChain(
            input_variables=["extracted_info"],
            output_variables=["retrieved_data"],
            transform=search_by_song_and_artist
        )
    elif input_type == "mood_or_theme":
        transform_chain = TransformChain(
            input_variables=["query"],
            output_variables=["retrieved_data"],
            transform=search_by_mood_or_theme
        )
    elif input_type == "specific_feature":
        transform_chain = TransformChain(
            input_variables=["query"],
            output_variables=["retrieved_data"],
            transform=search_by_specific_feature
        )
    else:
        raise ValueError(f"Unrecognized input type: {input_type}")

    sequential_chain = SequentialChain(
        chains=[llm_chain1, transform_chain, llm_chain2],
        input_variables=["query"],
        output_variables=["final_response"]
    )

    response = sequential_chain.invoke({"query": user_query})
    return response['final_response']

# 사용자 입력 예시
user_query_1 = "버즈의 가시나 박효신의 동경이라는 노래와 비슷한 노래를 추천해줘"
user_query_2 = "오늘 헤어져서 기분이 꿀꿀한데 부를 만한 노래 뭐있어?"
user_query_3 = "2000년대 남성 그룹들이 부른 신나는 노래 추천해줘"

# 실행
print(run_chain(user_query_1))
print(run_chain(user_query_2))
print(run_chain(user_query_3))


=== LLM Response ===
Type: song_and_artist
- Recommendation: [9023, 14421, 46771, 13334, 13334]

=== LLM Response ===
Type: mood_or_theme
- Recommendation: [12318, 31548, 60741, 67064, 67064]

=== LLM Response ===
Type: specific_feature
- Recommendation: [67921, 13792, 38516, 30395, 30395]


## 과거 LLM Recommend

In [ ]:
from langchain_milvus import Milvus
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_core.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate
from langchain.chains import SequentialChain, LLMChain
from langchain.chains.transform import TransformChain
from proto.langchainRecommend.langchainRecommend_pb2_grpc import LangchainRecommendServicer
from proto.langchainRecommend.langchainRecommend_pb2 import LangchainResponse, SimilarItem
import grpc
import traceback
import logging
import os

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class LangChainServiceGrpc(LangchainRecommendServicer):
    def __init__(self):
        # Load API keys from environment
        self.OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

        # Initialize LLM model
        self.llm = ChatOpenAI(
            temperature=0.5,
            max_tokens=4096,
            streaming=True,
            callbacks=[StreamingStdOutCallbackHandler()],
            model_name='gpt-4o-mini',
            api_key=self.OPENAI_API_KEY
        )

        # Embedding model for user profiles
        self.embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

        # Milvus 설정
        self.collection_name = "singsongsangsong_22286"
        self.vectorstore = Milvus(
            embedding_function=self.embedding_model,
            collection_name=self.collection_name,
            connection_args={"host": "localhost", "port": "19530"},
            text_field="song_name"
        )
        self.retriever = self.vectorstore.as_retriever()

    # 1. 노래 제목과 가수 이름 기반 검색 (유형 1)
    def search_by_song_and_artist(self, output):
        extracted_info = output["extracted_info"]

        artist_song_pairs = extracted_info.split(", ")
        search_results = []

        for pair in artist_song_pairs:
            artist_name = pair.split("Artist:")[-1].split(",")[0].strip()
            song_title = pair.split("Song:")[-1].strip()
            search_query = f"{artist_name} released the song '{song_title}'"
            results = self.retriever.get_relevant_documents(search_query)

            search_results += [doc.metadata['song_info_id'] for doc in results]

        search_results = self.ensure_five_recommendations(search_results)
        return {"retrieved_data": search_results}

    # 2. 기분이나 분위기 기반 검색 (유형 2)
    def search_by_mood_or_theme(self, user_query):
        prompt = f"""
        The user input is: "{user_query}". 
        Extract the mood or theme from the input and create a query to search for songs with similar emotional qualities, such as mood, energy level, and use case.
        """
        refined_query = self.llm(prompt)
        results = self.retriever.get_relevant_documents(refined_query.content)
        
        search_results = [doc.metadata['song_info_id'] for doc in results]
        search_results = self.ensure_five_recommendations(search_results)
        return {"retrieved_data": search_results}

    # 3. 특정 연도나 특성 기반 검색 (유형 3)
    def search_by_specific_feature(self, user_query):
        prompt = f"""
        The user is looking for songs with the following features: "{user_query}". 
        Create a detailed query that includes era, singer type, and genre to retrieve relevant songs from the database.
        """
        refined_query = self.llm(prompt)
        results = self.retriever.get_relevant_documents(refined_query.content)
        
        search_results = [doc.metadata['song_info_id'] for doc in results]
        search_results = self.ensure_five_recommendations(search_results)
        return {"retrieved_data": search_results}

    # 추천 목록을 정확히 5곡으로 맞추는 함수
    def ensure_five_recommendations(self, recommendation_list):
        while len(recommendation_list) < 5:
            recommendation_list.append(recommendation_list[-1])  # 마지막 곡을 반복 추가
        return recommendation_list[:5]

    # LLM을 이용해 입력 타입을 판단하는 함수
    def determine_input_type(self, user_query):
        prompt = f"""
        Classify the user's input into one of the following types:
        1. song_and_artist: If the user is asking for songs similar to specific songs or artists.
        2. mood_or_theme: If the user is describing a mood, theme, or situation to find songs that match those qualities.
        3. specific_feature: If the user is looking for songs based on specific features such as year, genre, or singer type.

        Input: "{user_query}"

        Your output should be in the following format:
        Type: <song_and_artist/mood_or_theme/specific_feature>
        """
        result = self.llm(prompt)
        input_type = result.content.lower().replace("type:", "").strip()
        return input_type

    # LLMChain을 통해 검색 및 추천 수행
    def run_chain(self, user_query):
        input_type = self.determine_input_type(user_query)

        if input_type == "song_and_artist":
            transform_chain = TransformChain(
                input_variables=["extracted_info"],
                output_variables=["retrieved_data"],
                transform=self.search_by_song_and_artist
            )
        elif input_type == "mood_or_theme":
            transform_chain = TransformChain(
                input_variables=["query"],
                output_variables=["retrieved_data"],
                transform=self.search_by_mood_or_theme
            )
        elif input_type == "specific_feature":
            transform_chain = TransformChain(
                input_variables=["query"],
                output_variables=["retrieved_data"],
                transform=self.search_by_specific_feature
            )
        else:
            raise ValueError(f"Unrecognized input type: {input_type}")

        prompt_extract = ChatPromptTemplate.from_template(
            """
            Extract the artist name and song title from the following query: {query}.
            If no song or artist is found, try to extract the mood, theme, or other musical characteristics from the query.

            Your output should follow this format:
            - Artist: <artist_name>
            - Song: <song_title> OR Mood/Theme: <mood_or_theme>
            """
        )

        prompt_refine = ChatPromptTemplate.from_template(
            """
            Based on the query "{query}" and the following song metadata retrieved from the database:
            {retrieved_data}

            Please provide exactly 5 song recommendations by their song info IDs, formatted as:

            - Recommendation: [<song_info_id1>, <song_info_id2>, <song_info_id3>, <song_info_id4>, <song_info_id5>]

            Only provide the song info IDs without any additional descriptions or explanations.
            """
        )

        llm_chain1 = LLMChain(llm=self.llm, prompt=prompt_extract, output_key="extracted_info")
        llm_chain2 = LLMChain(llm=self.llm, prompt=prompt_refine, output_key="final_response")

        sequential_chain = SequentialChain(
            chains=[llm_chain1, transform_chain, llm_chain2],
            input_variables=["query"],
            output_variables=["final_response"]
        )

        response = sequential_chain.invoke({"query": user_query})
        return response['final_response']

    def GetLangchainRecommendation(self, request, context):
        try:
            # 유사한 노래 검색 (song_info_id 리스트가 반환됨)
            search_results = self.run_chain(request.command)  # 유사한 노래 검색 (song_info_id 리스트)
            
            similar_items = []
            
            # 검색된 song_info_id 리스트를 순회하면서 각각의 노래 정보를 Milvus에서 조회
            for song_info_id in search_results:  # search_results는 [<song_info_id1>, <song_info_id2>, ...]
                logger.info(f"Fetching song details for song_info_id: {song_info_id}")

                # Milvus에서 song_info_id에 해당하는 곡 정보 조회
                query = f"song_info_id == {song_info_id}"  # song_info_id로 조회
                result = self.vectorstore.similarity_search(request.command, k=3, expr=query)  # 단일 결과 조회 (k=1)

                if not result:
                    logger.warning(f"No song found for song_info_id: {song_info_id}")
                    continue

                # 조회한 결과에서 메타데이터 추출
                song_metadata = result[0].metadata

                # SimilarItem 메시지 생성 및 추가
                similar_items.append(SimilarItem(
                    songInfoId=song_metadata.get("song_info_id"),  # song_info_id
                    songName=result[0].page_content,  # 노래 제목
                    singerName=song_metadata.get("artist_name"),  # 아티스트 이름
                    isMr=song_metadata.get("MR", False),  # MR 여부
                    ssss=song_metadata.get("ssss", ""),  # 추가 메타데이터 필드
                    audioFileUrl=song_metadata.get("audio_file_url", ""),  # 오디오 파일 URL
                    album=song_metadata.get("album", ""),  # 앨범 이름
                    songNumber=song_metadata.get("song_number", 0),  # 곡 번호
                    similarityScore=1.0  # 유사도 점수는 하드코딩했지만, 필요시 계산 가능
                ))

            # gRPC 응답 반환
            return LangchainResponse(similarItems=similar_items)

        except Exception as e:
            logger.error(f"Error during GetLangchainRecommendation: {e}")
            logger.error(traceback.format_exc())  # 전체 스택 트레이스를 출력합니다.
            context.set_code(grpc.StatusCode.INTERNAL)
            context.set_details("Internal server error")
            return LangchainResponse(similarItems=[])

In [2]:
pip install pymilvus

Note: you may need to restart the kernel to use updated packages.


## 콜렉션 삭제할때

In [16]:
from pymilvus import utility, connections

# Milvus에 연결
connections.connect(alias="new", host="www.dungdungcloud.shop", port="19530")

# 삭제할 컬렉션 이름
collection_name = "singsongsangsong_22286"

# 컬렉션이 존재하는지 확인한 후 삭제
if utility.has_collection(collection_name):
    utility.drop_collection(collection_name)
    print(f"Collection '{collection_name}' deleted successfully!")
else:
    print(f"Collection '{collection_name}' does not exist.")

# 남아 있는 컬렉션 목록 확인
collections = utility.list_collections()
print(f"Remaining collections: {collections}")

Collection 'singsongsangsong_22286' does not exist.
Remaining collections: []
